<a href="https://www.kaggle.com/code/savageopress12345/usfl-elos?scriptVersionId=126913789" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
yrs=[1983,1984,1985]
arr=[]
links=[]
for yr in yrs:
    html=requests.get("https://www.profootballarchives.com/"+str(yr)+"usfl.html")
    soup=BeautifulSoup(html.text)
    stats=soup.find("div",{"id":"stats"})
    table=stats.find_all("table")[0]
    for a in table.find_all("a"):
        links.append(a.get("href"))
        url="https://www.profootballarchives.com/"+str(a.get("href"))
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[4]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(yr)
            arr.append(h)
yrs=[2022]
links=[]
html=requests.get("https://www.profootballarchives.com/2022usflbir.html")
soup=BeautifulSoup(html.text)
table=soup.find_all("table")[3]
for a in table.find_all("a"):
    if '<a></a>' not in str(a):
        links.append(a.get("href"))
for link in links:
    for yr in yrs:
        
        link=str(link)
        if '83' in link:
            link=link.split("83")
            link=link[1]
        if '84' in link:
            link=link.split("84")
            link=link[1]
        if '85' in link:
            link=link.split("85")
            link=link[1]
        link=link
        url="https://www.profootballarchives.com/"+link
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[3]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(2022)
            arr.append(h)
df=pd.concat([pd.DataFrame(arr)])
df.columns=['Day','Date','VenueType','team2','score','result','OT','Location','Venue','Attendance','team1','season']
df=df.astype({'VenueType':'string'})

df=df[df.Date.notna()]
df=df[df.score.notna()]
df['score2']=0
df['score1']=0
for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    df.at[row.Index,'score1']=s1
    df.at[row.Index,'score2']=s2
df['Date']=df['Date'].str.replace("/","")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,"Date":"string"})  
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)


In [1]:
1

1

In [43]:
import math
import numpy as np
import pandas as pd
df=pd.read_csv("https://github.com/easella/usfl/raw/main/usflscores.csv")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,'Date':"string"})
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
hfa=[]
w=0

df.sort_values(by='Date',inplace=True)
l=0
gh={}
gh['g']=0
class Elo:
  global winnerHome
  def __init__(self,k,g=1,homefield = 182.5):
    self.ratingDict   = {}  
    self.k        = k
    self.g        = g
  def addPlayer(self,name,rating = 1500):
    self.ratingDict[name] = rating
  def gameOver(self, winner, loser, winnerHome,neutral,wins,loses):
    homef=gh['g']
    wb=0
    lb=0
    if winnerHome and neutral!='N':
      result = self.expectResult(self.ratingDict[winner] + homef, self.ratingDict[loser])
    if winnerHome==False and neutral!='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser] + homef)
    if neutral=='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser])
    wins=float(wins)
    loses=float(loses)
    if wins==loses:
        tie=True
    else:
        tie=False
    if tie:
        mult=(math.log(0+1))*(2.2/1*0.001+2.2)
        shift=(self.k*mult)*(0.5 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
    if tie !=True:
        win=eloLeague.ratingDict[winner]
        lose=eloLeague.ratingDict[loser]
        if neutral=='N':
          win=(eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome and neutral!='N':
          win=(homef+eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome!=True and neutral!='N':
          lose=(homef+eloLeague.ratingDict[loser])
          win=(eloLeague.ratingDict[winner])
        global zy
        if(win>lose):
          zy=(win-lose)
        if(win==lose):
          zy=(win-lose)
        if(win<lose):
          zy=(win-lose)
        mult=(math.log((wins-loses)+1))*(2.2/(zy)*0.001+2.2)
        shift=(self.k*mult)*(1 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
  def expectResult(self, p1, p2):
        global exp
        exp = (p2-p1)/400
        n2=10
        j=exp
        o=1/((n2**(j))+1)
        return float(o)
teams=set(df.team1.tolist()+df.team2.tolist())
ds={}
eloLeague=Elo(k=25)
currSeason=1983
q=pd.read_html("https://en.wikipedia.org/wiki/2023_USFL_season")[2]
q.columns = q.columns.droplevel(1)
q=q.replace("[a]","")
q=q.rename(columns={'Team[10]':'Team'})
q=q.rename(columns={'Home stadium':'Venue'})
venues={}
venues[2023]={}
for row in q.itertuples():
    l=list(row)
    team=str(l[1])
    team=team.replace("[a]","")
    team=team.replace(" ","")
    venues[2023][team]=row.Venue


    
q=pd.read_html("https://en.wikipedia.org/wiki/United_States_Football_League_(2022)#Teams")[2]
q.columns = q.columns.droplevel(1)
q=q.replace("[a]","")
q=q.rename(columns={'Team[72]':'Team'})
q=q.rename(columns={'Stadium':'Venue'})
venues[2022]={}
for row in q.itertuples():
    l=list(row)
    team=str(l[1])
    team=team.replace("[a]","")
    team=team.replace(" ","")
    team=team.replace("MemphisShowboats","TampaBayBandits")

    venues[2022][team]=row.Venue
t=pd.read_csv("https://ontheroadtovote.com/ncaab/usflscores.txt")
t=t.astype({'team1':"string",'team2':'string'})
t=t.replace("TampaBayBandits","MemphisShowboats")
df=pd.concat([df,t])
for row in df.itertuples():
    if row.season==2023:
        
        if str(row.team1)=='TampaBayBandits':
            df.at[row.Index,'team1']='MemphisShowboats'
        if str(row.team2)=='TampaBayBandits':
            df.at[row.Index,'team2']='MemphisShowboats'
for row in df.itertuples():
    if row.season>2021:
        if venues[row.season][row.team2]==venues[row.season][row.team1]:
            df.at[row.Index,'VenueType']='N'
df=df.drop(['Unnamed: 0'], axis=1)
df=df.drop_duplicates()
hfa=[]
df['score2']=0
df['score1']=0

for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    if row.season==2023:
        df.at[row.Index,'score1']=s2
        df.at[row.Index,'score2']=s1
    if row.season!=2023:
        df.at[row.Index,'score1']=s2
        df.at[row.Index,'score2']=s1
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,'Date':"string",'VenueType':'string'})
n=df[df.VenueType!='N']
hfaz=[]
for row in n.itertuples():
    if row.VenueType=='A':
        hfaz.append(row.score1-row.score2)
    else:
        hfaz.append(row.score2-row.score1)
homef=round(np.mean(hfaz))
for team in teams:
    eloLeague.addPlayer(team,rating=1500)
gh['g']=round(np.mean(hfaz)*25)
l=0
w=0

msqe={}
for i in range(10,50):
    msqe[i]=[]
s={}
ds[2023]={}
ds[2023]['North']={}
ds[2023]['South']={}
ds[2023]['North']['MichiganPanthers']=0
ds[2023]['North']['PhiladelphiaStars']=0
ds[2023]['North']['NewJerseyGenerals']=0
ds[2023]['North']['PittsburghMaulers']=0
ds[2023]['South']['BirminghamStallions']=0
ds[2023]['South']['NewOrleansBreakers']=0
ds[2023]['South']['HoustonGamblers']=0
ds[2023]['South']['TampaBayBandits']=0
ds[2022]={}
ds[2022]['North']={}
ds[2022]['South']={}
ds[2022]['North']['MichiganPanthers']=0
ds[2022]['North']['PhiladelphiaStars']=0
ds[2022]['North']['NewJerseyGenerals']=0
ds[2022]['North']['PittsburghMaulers']=0
ds[2022]['South']['BirminghamStallions']=0
ds[2022]['South']['NewOrleansBreakers']=0
ds[2022]['South']['HoustonGamblers']=0
ds[2022]['South']['TampaBayBandits']=0
for i in range(1983,1986):
    ds[i]={}
for i in range(1983,1986):
    
    dsdf=pd.read_html("https://www.profootballarchives.com/"+str(i)+"usfl.html")[2]
    
    dsdf.columns=dsdf.columns.droplevel(1)
    dsdf.columns=['Team','W','L','T','PCT','PF','PA','w','q','e','t','k','j','l','m']
    dsdf=dsdf.astype({'Team':'string','PCT':'string'})
    k=0
    for row in dsdf.itertuples():
        if k==0:
            div='ggg'
            ds[i][div]={}
        if 'PCT' in str(row.PCT):
            if str(row.Team) not in str(ds[i]):
                if k==0:
                    div='ggg'
                else:
                    div=row.Team
                ds[i][div]={}
        else:
            tea=str(row.Team)
            tea=tea.replace("*","")
            tea=tea.replace(" ","")
            if 'tea' not in str(ds[i]):
                ds[i][div][tea]=0
            else:
                ds[i][div][tea]=eloLeague.ratingDict[tea]
        k+=1
ds[1984]={'ggg': {'PhiladelphiaStars': 0,
  'NewJerseyGenerals': 0,
  'WashingtonFederals': 0,
  'PittsburghMaulers': 0},
 'Southern Division': {'BirminghamStallions': 0,
  'TampaBayBandits': 0,
  'NewOrleansBreakers': 0,
  'MemphisShowboats': 0,
  'JacksonvilleBulls': 0},
 'Central Division': {'HoustonGamblers': 0,
  'MichiganPanthers': 0,
  'SanAntonioGunslingers': 0,
  'OklahomaOutlaws': 0,
  'ChicagoBlitz': 0},
 'Pacific Division': {'LosAngelesExpress': 0,
  'ArizonaWranglers': 0,
  'DenverGold': 0,
  'OaklandInvaders': 0}}
test = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1dorlxpgaWZYvMqeCwi0fT4t04C1at9OdRglGFjkLfvw' +
                   '/export?gid=0&format=csv'
                  )
to={}
to[2022]={}
to[2023]={}
for row in test.Team.unique():
    to[2022][row]=0
    to[2023][row]=0
for row in test.itertuples():
    to[2022][row.Team]=row.t2022o
    to[2023][row.Team]=row.t2023o
for row in df.itertuples():
    if float(row.season)>currSeason:
        if row.season>2021:
            mean=[]
            for item in to[row.season].keys():
                mean.append(float(to[row.season][item])/100)
        for key in eloLeague.ratingDict.keys():
            
            if(eloLeague.ratingDict[key]!=1500):
                pre=eloLeague.ratingDict[key]
                if row.season>2021:
                    if key in str(to[row.season]):
                        tos=to[row.season][key]
                        tos=tos/100
                        if tos!=0:
                            veg=(-np.mean(mean)+float(tos))*-25
                    eloLeague.ratingDict[key]+=(veg*(2/3))
                        
                if row.season==2023 and key=='MemphisShowboats':
                    av=eloLeague.ratingDict[key]+eloLeague.ratingDict['MemphisShowboats']
                    av=av/2
                    eloLeague.ratingDict[key]=(av*(2/3))+(1505*(1/3))
                else:
                    eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1505*(1/3))
                af=eloLeague.ratingDict[key]
    
                
        currSeason+=1
    if row.score1>row.score2 or row.score1==row.score2:
        hfa=0
        if row.VenueType!='N':
            hfa=float(gh['g'])
        if row.VenueType=='A':

            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1]+hfa)<(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=True
            
        if row.VenueType==('H' or 'N'):
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=False
        eloLeague.gameOver(row.team1,row.team2,winnerhome,row.VenueType,row.score1,row.score2)
    if row.score1<row.score2:
        hfa=0
        if row.VenueType!='N':
            hfa=gh['g']
        if row.VenueType=='A':
            if (eloLeague.ratingDict[row.team1]+hfa)<(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=False
        if row.VenueType=='H':
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=True
        if row.VenueType=='N':
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=True
        eloLeague.gameOver(row.team2,row.team1,winnerhome,row.VenueType,row.score2,row.score1)
    from sklearn.metrics import mean_squared_error
    if row.VenueType =='N' or row.VenueType=='H':
        yt = [row.score2-row.score1]
        pred=(eloLeague.ratingDict[row.team2]+hfa)-eloLeague.ratingDict[row.team1]
    else:
        yt = [row.score1-row.score2]
        pred=(eloLeague.ratingDict[row.team1]+hfa)-eloLeague.ratingDict[row.team2]
    for i in range(10,50):
        y_pred=[round((pred)/i)]
        msqe[i].append(mean_squared_error(yt, y_pred))
css=df[df.season==2023]
if len(css)==0:
    for key in eloLeague.ratingDict.keys():
            
        if(eloLeague.ratingDict[key]!=1500):
            pre=eloLeague.ratingDict[key]
            eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1500*(1/3))
            af=eloLeague.ratingDict[key]
ms={}
for i in range(10,50):
    ms[i]=0
for key in msqe.keys():
    ms[key]=np.mean(msqe[key])
mas={k: v for k, v in sorted(ms.items(), key=lambda item: item[1],reverse=False)[:1]}
for key in mas.keys():
    print("best value for spreads:"+str(key))  

best value for spreads:22


In [ ]:
{k: v for k, v in sorted(ms.items(), key=lambda item: item[1],reverse=False)[:1]}

In [17]:
eloLeague.ratingDict['BirminghamStallions']+gh['g']

1602.613163895251

In [12]:
1602.61-1485.26

117.34999999999991

In [18]:
eloLeague.ratingDict['TampaBayBandits']

1500.0000175699456

In [8]:
1675.91-1360.53

315.3800000000001

In [13]:
117/22

5.318181818181818

In [ ]:
(1629.1534470621875-1565.1224714185287)/22

In [ ]:
eloLeague.ratingDict['PittsburghMaulers']

In [22]:
print(100/(w+l)*w)

64.90066225165563


In [19]:
l2s={k: v for k, v in sorted(eloLeague.ratingDict.items(), key=lambda item: item[1],reverse=True)}
for key in l2s.keys():
    print(key,eloLeague.ratingDict[key])

NewOrleansBreakers 1609.9176376180574
MichiganPanthers 1547.0502534463308
BirminghamStallions 1536.613163895251
PittsburghMaulers 1518.3582768257177
PhiladelphiaStars 1505.7062904637494
LosAngelesExpress 1500.000068391022
OklahomaOutlaws 1500.0000473501016
WashingtonFederals 1500.0000393213609
SanAntonioGunslingers 1500.0000391269832
PortlandBreakers 1500.000034231164
OrlandoRenegades 1500.0000231719941
ChicagoBlitz 1500.0000226258617
TampaBayBandits 1500.0000175699456
DenverGold 1500.0000113963956
ArizonaOutlaws 1499.999999682102
BostonBreakers 1499.9999882275706
JacksonvilleBulls 1499.9999742340972
ArizonaWranglers 1499.9999664561815
OaklandInvaders 1499.999936946667
BaltimoreStars 1499.9999255661667
MemphisShowboats 1484.2601969397947
NewJerseyGenerals 1437.5569021821173
HoustonGamblers 1360.537184331363
